In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/probabilistic-forecasting-i-temperature/sample_submission.csv
/kaggle/input/probabilistic-forecasting-i-temperature/train.csv
/kaggle/input/probabilistic-forecasting-i-temperature/test.csv


In [2]:
pip install statsforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 5.0 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.2.0-py3-none-any.whl size=158196 sha256=cfabb7e12005202651a9821eba87c3e33fe2d9491542e2cb

In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsforecast import StatsForecast

import plotly.graph_objects as go

import plotly.express as px

import plotly.graph_objects as go

import warnings

from tqdm import TqdmExperimentalWarning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.signal
import math
from colorama import Fore, Style

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)

/opt/conda/lib/python3.10/site-packages/statsforecast/core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
# Importing the data

import pandas as pd
train = pd.read_csv("/kaggle/input/probabilistic-forecasting-i-temperature/train.csv")
train  

,id,date,feature_AA,feature_AB,feature_BA,feature_BB,feature_CA,feature_CB,Temperature
0,0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,1,2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401,30.459999
2,2,2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310,30.038000
3,3,2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310,27.013000
4,4,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
...,...,...,...,...,...,...,...,...,...
64315,64315,2018-05-01 22:45:00,12.458,1.407,7.889,-0.746,4.843,1.523,10.622000
64316,64316,2018-05-01 23:00:00,14.200,1.541,9.310,0.284,4.752,1.249,10.482000
64317,64317,2018-05-01 23:15:00,12.659,1.474,7.924,-0.959,4.569,1.310,10.271000
64318,64318,2018-05-01 23:30:00,13.329,1.541,8.671,-0.533,4.630,1.371,10.341000


In [5]:
# Importing the data
import pandas as pd
test = pd.read_csv("/kaggle/input/probabilistic-forecasting-i-temperature/test.csv")  
test

,id,date,feature_AA,feature_AB,feature_BA,feature_BB,feature_CA,feature_CB
0,64320,2018-05-02 00:00:00,14.133,2.076,9.346,-0.036,4.538,1.279
1,64321,2018-05-02 00:15:00,12.123,1.541,7.818,-0.746,4.538,1.249
2,64322,2018-05-02 00:30:00,12.525,1.741,8.209,-0.284,4.477,1.279
3,64323,2018-05-02 00:45:00,12.324,1.942,7.747,0.249,4.630,1.371
4,64324,2018-05-02 01:00:00,14.602,1.407,9.630,-0.320,4.477,1.310
...,...,...,...,...,...,...,...,...
5355,69675,2018-06-26 18:45:00,9.310,3.550,5.437,1.670,3.868,1.462
5356,69676,2018-06-26 19:00:00,10.114,3.550,6.183,1.564,3.716,1.462
5357,69677,2018-06-26 19:15:00,10.784,3.349,7.000,1.635,3.746,1.432
5358,69678,2018-06-26 19:30:00,11.655,3.617,7.533,1.706,4.173,1.523


In [6]:
# Importing the data
import pandas as pd
submission = pd.read_csv("/kaggle/input/probabilistic-forecasting-i-temperature/sample_submission.csv")
submission

,id,0.025,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,...,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,0.975
0,64320,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,64321,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,64322,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,64323,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,64324,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,69675,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5356,69676,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5357,69677,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5358,69678,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Length of forecast period
horizon = len(test)
print(f"Forecast horizon is {horizon}")

Forecast horizon is 5360


In [8]:
train.set_index('date', inplace=True)

In [9]:
# Drop column named id
train.drop(columns = 'id', inplace = True)

In [10]:
# Selecting a subset of columns containing "feature" in data frame "train" and assigning them to a new data frame named "X_train"
features_columns = [col for col in train.columns if 'feature' in col]
X_train = train[features_columns]
X_train.head()

,feature_AA,feature_AB,feature_BA,feature_BB,feature_CA,feature_CB
date,,,,,,
2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340
2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401
2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310
2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310
2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371


In [11]:
# Selecting only the columns in the "features_columns" list from the "test" dataframe and assigning them to a new dataframe called "X_test"
X_test = test[features_columns]
X_test.head()

,feature_AA,feature_AB,feature_BA,feature_BB,feature_CA,feature_CB
0,14.133,2.076,9.346,-0.036,4.538,1.279
1,12.123,1.541,7.818,-0.746,4.538,1.249
2,12.525,1.741,8.209,-0.284,4.477,1.279
3,12.324,1.942,7.747,0.249,4.630,1.371
4,14.602,1.407,9.630,-0.320,4.477,1.310


In [12]:
# Changing the index ("index") of the "Temperature" column in the "train" data frame to the index of the "X_train" data frame
# train['Temperature'].index = X_train.index

In [13]:
# Replacing the index ("index") of data frame "X_train" with the index of column "Temperature" of data frame "train"
# X_train.index = train['Temperature'].index

In [14]:
%%time
# Define the SARIMAX model
model = sm.tsa.SARIMAX(endog = train['Temperature'], exog=X_train, order=(1, 0, 1), seasonal_order=(1, 0, 1, 4))

# Fit the model
results = model.fit()

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.07653D+00    |proj g|=  8.06607D-01


 This problem is unconstrained.



At iterate    5    f=  1.29946D+00    |proj g|=  1.13810D+00

At iterate   10    f=  8.01305D-01    |proj g|=  6.75998D-01

At iterate   15    f=  6.72119D-01    |proj g|=  1.86089D-01

At iterate   20    f=  6.36427D-01    |proj g|=  3.93149D-02

At iterate   25    f=  6.35327D-01    |proj g|=  1.82564D-02

At iterate   30    f=  6.34155D-01    |proj g|=  1.57171D-02

At iterate   35    f=  6.34065D-01    |proj g|=  6.91080D-03

At iterate   40    f=  6.33359D-01    |proj g|=  5.97330D-02

At iterate   45    f=  6.28749D-01    |proj g|=  5.86226D-02

At iterate   50    f=  6.24565D-01    |proj g|=  2.41351D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


CPU times: user 5min 5s, sys: 2min 56s, total: 8min 1s
Wall time: 3min 33s


In [15]:
# In-sample predictions
train['forecast'] = results.predict(start=train.index[0], end=train.index[-1], dynamic=False, exog=X_train)

# Out-of-sample predictions
# Assuming you have a DataFrame `future_X` with future values of your external regressors
forecast = results.predict(start=test.index[0], end=test.index[-1], exog=X_test)

In [16]:
train.head()

,feature_AA,feature_AB,feature_BA,feature_BB,feature_CA,feature_CB,Temperature,forecast
date,,,,,,,,
2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000,0.105788
2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401,30.459999,30.504085
2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310,30.038000,30.420771
2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310,27.013000,29.970276
2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001,26.719721


In [17]:
quantiles = [0.025, 0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,0.975]

In [18]:
# Selecting the second half of the quantiles array and inverting it
levels = np.array([95, 90, 80, 70, 60, 50, 40, 30, 20, 10])
levels

array([95, 90, 80, 70, 60, 50, 40, 30, 20, 10])

In [19]:
# Making predictions in specified steps using the get_forecast() method of the SARIMAX model and calculating the confidence intervals of the predicted values
forecast_results = results.get_forecast(steps=horizon, exog=X_test, alpha=0.1)  
conf_int = forecast_results.conf_int()
conf_int

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


,lower Temperature,upper Temperature
2018-05-02 00:00:00,9.981731,11.743317
2018-05-02 00:15:00,9.568438,12.171302
2018-05-02 00:30:00,9.237481,12.466913
2018-05-02 00:45:00,8.940044,12.690545
2018-05-02 01:00:00,8.700082,12.910293
...,...,...
2018-06-26 18:45:00,-20.039076,20.179655
2018-06-26 19:00:00,-20.048708,20.170023
2018-06-26 19:15:00,-20.051897,20.166834
2018-06-26 19:30:00,-20.043102,20.175629


In [20]:
fcst = pd.DataFrame(index=test['date'])

# Add mean forecast to the DataFrame
fcst['SARIMAX'] = results.get_forecast(steps=horizon, exog=X_test).predicted_mean
fcst.head()

,SARIMAX
date,
2018-05-02 00:00:00,10.862524
2018-05-02 00:15:00,10.869870
2018-05-02 00:30:00,10.852197
2018-05-02 00:45:00,10.815295
2018-05-02 01:00:00,10.805187


In [21]:
# Specify your confidence levels
levels = np.array([95, 90, 80, 70, 60, 50, 40, 30, 20, 10])

# Iterate over the specified confidence levels to compute and store intervals
for level in levels:
    alpha = (100 - level) / 100
    forecast_results = results.get_forecast(steps=horizon, exog=X_test, alpha=alpha)  
    conf_int = forecast_results.conf_int()

    fcst[f'SARIMAX-lo-{level}'] = conf_int['lower Temperature'].values
    fcst[f'SARIMAX-hi-{level}'] = conf_int['upper Temperature'].values

# Reset index to add 'ds' column with datetime
fcst.reset_index(inplace=True)
fcst.rename(columns={'index': 'date'}, inplace=True)
fcst = pd.DataFrame(index=test['date'])

# Add mean forecast to the DataFrame
fcst['SARIMAX'] = results.get_forecast(steps=horizon, exog=X_test).predicted_mean

# Specify your confidence levels
levels = np.array([95, 90, 80, 70, 60, 50, 40, 30, 20, 10])

# Iterate over the specified confidence levels to compute and store intervals
for level in levels:
    alpha = (100 - level) / 100
    forecast_results = results.get_forecast(steps=horizon, exog=X_test, alpha=alpha)  
    conf_int = forecast_results.conf_int()

    fcst[f'SARIMAX-lo-{level}'] = conf_int['lower Temperature']
    fcst[f'SARIMAX-hi-{level}'] = conf_int['upper Temperature']

# Reset index to add 'ds' column with datetime
fcst.reset_index(inplace=True)
fcst.rename(columns={'index': 'date'}, inplace=True)

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword argu

In [22]:
fcst.head()

,date,SARIMAX,SARIMAX-lo-95,SARIMAX-hi-95,SARIMAX-lo-90,SARIMAX-hi-90,SARIMAX-lo-80,SARIMAX-hi-80,SARIMAX-lo-70,SARIMAX-hi-70,...,SARIMAX-lo-50,SARIMAX-hi-50,SARIMAX-lo-40,SARIMAX-hi-40,SARIMAX-lo-30,SARIMAX-hi-30,SARIMAX-lo-20,SARIMAX-hi-20,SARIMAX-lo-10,SARIMAX-hi-10
0,2018-05-02 00:00:00,10.862524,9.981731,11.743317,9.981731,11.743317,9.981731,11.743317,9.981731,11.743317,...,9.981731,11.743317,9.981731,11.743317,9.981731,11.743317,9.981731,11.743317,9.981731,11.743317
1,2018-05-02 00:15:00,10.869870,9.568438,12.171302,9.568438,12.171302,9.568438,12.171302,9.568438,12.171302,...,9.568438,12.171302,9.568438,12.171302,9.568438,12.171302,9.568438,12.171302,9.568438,12.171302
2,2018-05-02 00:30:00,10.852197,9.237481,12.466913,9.237481,12.466913,9.237481,12.466913,9.237481,12.466913,...,9.237481,12.466913,9.237481,12.466913,9.237481,12.466913,9.237481,12.466913,9.237481,12.466913
3,2018-05-02 00:45:00,10.815295,8.940044,12.690545,8.940044,12.690545,8.940044,12.690545,8.940044,12.690545,...,8.940044,12.690545,8.940044,12.690545,8.940044,12.690545,8.940044,12.690545,8.940044,12.690545
4,2018-05-02 01:00:00,10.805187,8.700082,12.910293,8.700082,12.910293,8.700082,12.910293,8.700082,12.910293,...,8.700082,12.910293,8.700082,12.910293,8.700082,12.910293,8.700082,12.910293,8.700082,12.910293


In [23]:
# Assuming model is defined as follows, with the first element being the model name you want to use
model = 'SARIMAX'

quantile_mapping = {
    95: ['0.025', '0.975'],
    90: ['0.05', '0.95'],
    80: ['0.10', '0.90'],
    70: ['0.15', '0.85'],
    60: ['0.20', '0.80'],
    50: ['0.25', '0.75'],
    40: ['0.30', '0.70'],
    30: ['0.35', '0.65'],
    20: ['0.40', '0.60'],
    10: ['0.45', '0.55']
}

for level, q_pair in quantile_mapping.items():
    lo_col = f'{model}-lo-{level}' 
    hi_col = f'{model}-hi-{level}' 

    submission[q_pair[0]] = fcst[lo_col].values 
    submission[q_pair[1]] = fcst[hi_col].values 

# Directly copy model[0] to '0.50' quantile column
submission['0.50'] = fcst[model].values

In [24]:
submission.head()

,id,0.025,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,...,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,0.975
0,64320,9.981731,9.981731,9.981731,9.981731,9.981731,9.981731,9.981731,9.981731,9.981731,...,11.743317,11.743317,11.743317,11.743317,11.743317,11.743317,11.743317,11.743317,11.743317,11.743317
1,64321,9.568438,9.568438,9.568438,9.568438,9.568438,9.568438,9.568438,9.568438,9.568438,...,12.171302,12.171302,12.171302,12.171302,12.171302,12.171302,12.171302,12.171302,12.171302,12.171302
2,64322,9.237481,9.237481,9.237481,9.237481,9.237481,9.237481,9.237481,9.237481,9.237481,...,12.466913,12.466913,12.466913,12.466913,12.466913,12.466913,12.466913,12.466913,12.466913,12.466913
3,64323,8.940044,8.940044,8.940044,8.940044,8.940044,8.940044,8.940044,8.940044,8.940044,...,12.690545,12.690545,12.690545,12.690545,12.690545,12.690545,12.690545,12.690545,12.690545,12.690545
4,64324,8.700082,8.700082,8.700082,8.700082,8.700082,8.700082,8.700082,8.700082,8.700082,...,12.910293,12.910293,12.910293,12.910293,12.910293,12.910293,12.910293,12.910293,12.910293,12.910293


In [25]:
submission.to_csv('submission.csv', index=False)